Step 1. 데이터 다운로드

$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics

Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!


In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you"]


Step 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.


In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해)
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os


file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
   raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.

print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?


['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [3]:
for idx, sentence in enumerate(raw_corpus):
   if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
   if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

   if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
      
   print(sentence)


Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [4]:
def preprocess_sentence(sentence):
   sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
   sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
   sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
   sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

   sentence = sentence.strip()

   sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
  
   return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.


<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
   if len(sentence) == 0: continue
   if sentence[-1] == ":": continue
      
   corpus.append(preprocess_sentence(sentence))
      
corpus[:10]


['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

Step 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!


In [6]:
#tokenize() 함수로 데이터를 Tensor로 변환한 후, 


In [7]:
def tokenize(corpus):
   # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
   tokenizer = tf.keras.preprocessing.text.Tokenizer(
       num_words=12000,  # 전체 단어의 개수
       filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
       oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
   )
   tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

   # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
   tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

   # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
   # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
   tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre') 

   print(tensor,tokenizer)
   return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   0    0    0 ...  110    5    3]
 [   0    0    0 ...  110    5    3]
 [   0    0    0 ... 3034   15    3]
 ...
 [   0    0    0 ... 4553    4    3]
 [   0    0    0 ... 2965  132    3]
 [   0    0    0 ... 1787    5    3]] <keras_preprocessing.text.Tokenizer object at 0x7f1288396910>


In [8]:
print(tensor[:3, :10])

[[ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2 11]]


In [9]:
for idx in tokenizer.index_word:
   print(idx, ":", tokenizer.index_word[idx])

   if idx >= 10: break


1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [10]:
#sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.


from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,train_size = 0.8)


print(src_input[0])
print(tgt_input[0])


[  0   0   0   0   0   0   0   0   0   2 143  40 933 140 591   4 124  24
 110   5]
[  0   0   0   0   0   0   0   0   2 143  40 933 140 591   4 124  24 110
   5   3]


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

4-2. 인공지능 학습하기

In [12]:
class TextGenerator(tf.keras.Model):
   def __init__(self, vocab_size, embedding_size, hidden_size):
       super(TextGenerator, self).__init__()
      
       self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
       self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
       self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
       self.linear = tf.keras.layers.Dense(vocab_size)
      
   def call(self, x):
       out = self.embedding(x)
       out = self.rnn_1(out)
       out = self.rnn_2(out)
       out = self.linear(out)
      
       return out
  
embedding_size = 4012
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

print('fin')


fin


In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

print('fin')


fin


In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  48148012  
_________________________________________________________________
lstm (LSTM)                  multiple                  20631552  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 89,473,293
Trainable params: 89,473,293
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)


Epoch 1/10
93/93 [==============================] - 46s 497ms/step - loss: 3.4030
Epoch 2/10
93/93 [==============================] - 46s 500ms/step - loss: 2.8095
Epoch 3/10
93/93 [==============================] - 47s 502ms/step - loss: 2.6937
Epoch 4/10
93/93 [==============================] - 47s 502ms/step - loss: 2.5779
Epoch 5/10
93/93 [==============================] - 47s 501ms/step - loss: 2.5036
Epoch 6/10
93/93 [==============================] - 47s 501ms/step - loss: 2.5749
Epoch 7/10
93/93 [==============================] - 47s 505ms/step - loss: 2.4326
Epoch 8/10
93/93 [==============================] - 47s 510ms/step - loss: 2.3536
Epoch 9/10
93/93 [==============================] - 48s 512ms/step - loss: 2.3353
Epoch 10/10
93/93 [==============================] - 48s 513ms/step - loss: 2.2557


Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!


In [16]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=15):
   # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
   test_input = tokenizer.texts_to_sequences([init_sentence])
   test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
   end_token = tokenizer.word_index["<end>"]

   # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
   while True:
       predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다.
       predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.

       # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다.
       test_tensor = tf.concat([test_tensor,
                                                tf.expand_dims(predict_word, axis=0)], axis=-1)

       # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
       if predict_word.numpy()[0] == end_token: break
       if test_tensor.shape[1] >= max_len: break

   generated = ""
   # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
   for word_index in test_tensor[0].numpy():
       generated += tokenizer.index_word[word_index] + " "

   return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

print('fin')


fin


In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)


'<start> i love . <end> '